In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def kpss_veri_cek(yil: int, donem: int, max_sayfa: int = 16):
    """
    Belirtilen yıl ve dönemdeki KPSS atama verilerini memurlar.net'ten çeker.

    Args:
        yil (int): Verilerin çekileceği yıl (örn: 2024).
        donem (int): Atama dönemi (1 veya 2).
        max_sayfa (int): Taranacak maksimum sayfa sayısı.

    Returns:
        pandas.DataFrame: Çekilen verileri içeren DataFrame.
    """
    
    # Her sütun için boş listeler oluşturulur
    program_kodlari = []
    kurum_adlari = []
    kadro_unvanlari = []
    iller = []
    kontenjanlar = []
    nitelikler_listesi = []
    bos_kontenjanlar = []
    min_puanlar = []
    max_puanlar = []

    print(f"--- {yil}/{donem} KPSS verileri çekilmeye başlanıyor ---")

    for sayfa in range(1, max_sayfa + 1):
        # URL, fonksiyona verilen yıl ve dönem parametreleri ile dinamik olarak oluşturulur
        URL = f'https://kpss.memurlar.net/kpssrobotu/{yil}/{donem}/{sayfa}.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode='

        print(f"{sayfa}. sayfa taranıyor: {URL}")
        
        try:
            response = requests.get(URL, timeout=10) # Zaman aşımı eklemek iyi bir pratiktir
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Tablodaki satırları seç
            rows = soup.select('table.KpssList tr')[1:]
            
            # Eğer sayfa boşsa veya veri yoksa döngüyü sonlandır
            if not rows:
                print(f"{sayfa}. sayfada veri bulunamadı. Veri çekme işlemi tamamlandı.")
                break

            for row in rows:
                cols = row.find_all('td')
                if len(cols) < 10:
                    continue
                
                text = cols[2].text.strip()

                pattern = re.compile(r'\d+')

                match = pattern.search(text)

                if match:
                    split_index = match.start()
    
                oncesi = text[:split_index]
                sonrasi = text[split_index:]

                program_kodlari.append(cols[1].text.strip())
                kurum_adlari.append(oncesi)
                nitelikler_listesi.append(sonrasi)
                kadro_unvanlari.append(cols[len(cols)-6].text.strip())
                iller.append(cols[len(cols)-5].text.strip())
                kontenjanlar.append(cols[len(cols)-4].text.strip())
                bos_kontenjanlar.append(cols[len(cols)-3].text.strip())
                min_puanlar.append(cols[len(cols)-2].text.strip())
                max_puanlar.append(cols[len(cols)-1].text.strip())

        except requests.exceptions.RequestException as e:
            print(f"Hata oluştu: {e}. Bu sayfa atlanıyor.")
            continue
    
    # Toplanan listeler kullanılarak DataFrame oluşturulur
    df = pd.DataFrame({
        'Program Kodu': program_kodlari,
        'Kurum Adı': kurum_adlari,
        'Ünvanı': kadro_unvanlari,
        'Kadro Detayları': nitelikler_listesi,
        'İl': iller,
        'Kontenjan': kontenjanlar,
        'Boş Kontenjan': bos_kontenjanlar,
        'Min Puan': min_puanlar,
        'Max Puan': max_puanlar,
    })
    
    print(f"\nToplam {len(df)} adet kadro bilgisi çekildi.")
    return df

# --- KODU KULLANMA ÖRNEĞİ ---

# Örnek 1: 2024 yılı 2. atama verilerini çekmek için
# yil = 2024
# donem = 2
# kpss_df = kpss_veri_cek(yil, donem)


# Örnek 2: 2025 yılı 1. atama verilerini çekmek için
yil = 2025
donem = 1
kpss_df = kpss_veri_cek(yil, donem)


# DataFrame'in boş olup olmadığını kontrol et
if not kpss_df.empty:
    # Verileri ekrana yazdırma
    pd.set_option('display.max_rows', 100) # Ekrana 100 satır yazdır
    print("\n--- Çekilen Verilerin Önizlemesi ---")
    print(kpss_df.head()) # İlk 5 satırı göster

# Verileri bir CSV dosyasına kaydetme
# Dosya adının başına /tmp/ ekleyerek yazılabilir bir konuma kaydediyoruz.
    dosya_adi = f'/Users/gp/Desktop/kpss-veri-cekme/kpss_{yil}_{donem}_kadrolari.csv' 
    kpss_df.to_csv(dosya_adi, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{dosya_adi}' dosyasına kaydedildi.")  
else:
    print("\nİlgili dönem için herhangi bir veri çekilemedi.")

--- 2025/1 KPSS verileri çekilmeye başlanıyor ---
1. sayfa taranıyor: https://kpss.memurlar.net/kpssrobotu/2025/1/1.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode=
2. sayfa taranıyor: https://kpss.memurlar.net/kpssrobotu/2025/1/2.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode=
3. sayfa taranıyor: https://kpss.memurlar.net/kpssrobotu/2025/1/3.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode=
4. sayfa taranıyor: https://kpss.memurlar.net/kpssrobotu/2025/1/4.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode=
5. sayfa taranıyor: https://kpss.memurlar.net/kpssrobotu/2025/1/5.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode=
6. sayfa taranıyor: https://kpss.memurlar.net/kpssrobotu/2025/1/6.sayfa?SelectedItem=Kadro+Arama&BranchType=4&BranchCode=&Organization=&JobTitle=&CityCode=
7. sayfa taran